In [1]:
from datasets import load_dataset

example = load_dataset("dohuyen/9k-questions", split="train[:1]")[0]
dataset = load_dataset("dohuyen/9k-questions", split="train[1:101]")

display(dataset)
display(example)

Dataset({
    features: ['Answer', 'ID', 'Question', 'Choices'],
    num_rows: 100
})

{'Answer': 'C. 4',
 'ID': '1',
 'Question': 'Mỗi phân tử kháng thể IgG đơn phân trong huyết thanh có bao nhiêu chuỗi polypeptide ',
 'Choices': ['A. 1', 'B. 2', 'C. 4', 'D. 10']}

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "microsoft/Phi-3-mini-4k-instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
def compile_messages(sample: dict, add_context: bool = False) -> list[dict]:
    user_message = "{question}\n\n{choices}".format(
        question=sample["Question"],
        choices="\n".join(sample["Choices"])
    )
    if add_context:
        user_message = (
            "You are a helpful Vietnamese medical AI assistant. "
            "Answer the following multiple choice questions in Vietnamese "
            "regarding medical scenarios in the following format:\n\n"
        ) + user_message
    messages = [{"role": "user", "content": user_message}]

    if add_context:
        messages.append({"role": "assistant", "content": sample["Answer"]})

    return messages

example_messages = compile_messages(example, add_context=True)
print(tokenizer.apply_chat_template(example_messages, tokenize=False))

<|user|>
You are a helpful Vietnamese medical AI assistant. Answer the following multiple choice questions in Vietnamese regarding medical scenarios in the following format:

Mỗi phân tử kháng thể IgG đơn phân trong huyết thanh có bao nhiêu chuỗi polypeptide 

A. 1
B. 2
C. 4
D. 10<|end|>
<|assistant|>
C. 4<|end|>
<|endoftext|>


In [4]:
def generate_answer(sample: dict) -> str:
    messages = example_messages + compile_messages(sample)

    prompt = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            do_sample=True,
            temperature=0.001,
            top_p=0.999,
        )

    return tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)

display(generate_answer(dataset[0]))

'C. 5'

In [5]:
def generate_prompt(sample) -> str:
    messages = example_messages + compile_messages(sample)
    return tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

example_indexed_prompts = sorted(
    enumerate(map(generate_prompt, dataset.select(range(3)))),
    key=lambda i_pmpt: len(i_pmpt[1])
)

for i, prompt in example_indexed_prompts:
    print(f"{i}: {prompt}\n")

0: <|user|>
You are a helpful Vietnamese medical AI assistant. Answer the following multiple choice questions in Vietnamese regarding medical scenarios in the following format:

Mỗi phân tử kháng thể IgG đơn phân trong huyết thanh có bao nhiêu chuỗi polypeptide 

A. 1
B. 2
C. 4
D. 10<|end|>
<|assistant|>
C. 4<|end|>
<|user|>
Một phân tử IgM trong huyết thanh có mấy vị trí kết hợp kháng nguyên 

A. 1
B. 2
C. 5
D. 10<|end|>
<|assistant|>


2: <|user|>
You are a helpful Vietnamese medical AI assistant. Answer the following multiple choice questions in Vietnamese regarding medical scenarios in the following format:

Mỗi phân tử kháng thể IgG đơn phân trong huyết thanh có bao nhiêu chuỗi polypeptide 

A. 1
B. 2
C. 4
D. 10<|end|>
<|assistant|>
C. 4<|end|>
<|user|>
Tế bào sản xuất kháng thể là 

A. lympho bào B
B. lympho bào T
C. tế bào plasma (tuơng bào, plasmocyte)
D. đại thực bào<|end|>
<|assistant|>


1: <|user|>
You are a helpful Vietnamese medical AI assistant. Answer the following mult

In [6]:
from more_itertools import unzip

_, example_prompts = unzip(example_indexed_prompts)
example_batch = tokenizer(list(example_prompts), padding=True, return_tensors="pt")
display(example_batch["input_ids"])
display(example_batch["attention_mask"])

tensor([[32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
         32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
         32000, 32000, 32000, 32000, 32000, 32010,   887,   526,   263,  8444,
         18444,   968, 16083,   319, 29902, 20255, 29889,   673,   278,  1494,
          2999,  7348,  5155,   297, 18444,   968, 11211, 16083, 21846,   297,
           278,  1494,  3402, 29901,    13,    13, 29924,   228,   190,   154,
         29875,  1374, 10031,   260,   228,   190,   176,   413, 19990,   865,
           266, 31957, 12815, 29954, 29871, 30128, 30556, 29876,  1374, 10031,
           534,   549,   298,  8631, 30717, 29873,  1135, 29882, 28810,   289,
          6241,   302,  2918, 30037, 29884,   521, 29884,   228,   190,   154,
         29875,  1248,   668,   415,   680, 29871,    13,    13, 29909, 29889,
         29871, 29896,    13, 29933, 29889, 29871, 29906,    13, 29907, 29889,
         29871, 29946,    13, 29928, 29889, 29871, 2

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [7]:
example_batch["input_ids"].shape

torch.Size([3, 201])

In [8]:
example_batch_len = example_batch["input_ids"].shape[1]

with torch.no_grad():
    batch_outputs = model.generate(
        **example_batch.to(model.device),
        max_new_tokens=100,
        do_sample=True,
        temperature=0.001,
        top_p=0.999,
    )

for (j, _), output in zip(example_indexed_prompts, batch_outputs):
    print(f"{j}:", end=" ")
    print(tokenizer.decode(output[example_batch_len:], skip_special_tokens=True))

0: C. 5
2: C. tế bào plasma (tuơng bào, plasmocyte)
1: B. 5


Để giải thích điều này, ta sẽ xem xét cấu trúc của IgM, một loài nhân vi sinh thị không thể được biểu diễn bởi một phân tử IgM hoàn chỉnh.


In [9]:
from more_itertools import unzip
from tqdm import tqdm

def generate_answers_batched(dataset, batch_size=8):
    all_predictions = dict[int, str]()
    indexed_prompts = sorted(
        enumerate(map(generate_prompt, dataset)),
        key=lambda i_prompt: len(i_prompt[1])
    )

    for i in tqdm(range(0, len(indexed_prompts), batch_size), desc="Batch"):
        batch_idxs, batch_prompts = unzip(indexed_prompts[i: i + batch_size])
        batch_idxs, batch_prompts = list(batch_idxs), list(batch_prompts)

        batch_inputs = tokenizer(batch_prompts, padding=True, return_tensors="pt")
        batch_input_length = batch_inputs["input_ids"].shape[1]

        with torch.no_grad():
            batch_outputs = model.generate(
                **batch_inputs.to(model.device),
                max_new_tokens=100,
                do_sample=True,
                temperature=0.001,
                top_p=0.999,
            )

        for j, output in zip(batch_idxs, batch_outputs):
            all_predictions[j] = tokenizer.decode(
                output[batch_input_length:], skip_special_tokens=True
            )

    return dataset.add_column(
        "Prediction", [all_predictions[i] for i in range(len(dataset))]
    )

example_preds = generate_answers_batched(dataset.select(range(12)), batch_size=4)
display(example_preds["Prediction"])

Batch: 100%|██████████| 3/3 [00:09<00:00,  3.13s/it]


['C. 5',
 'B. 5\n\n\nĐể giải thích điều này, ta sẽ xem xét cấu trúc của IgM, một loài nhân vi sinh thị không thể được biểu diễn bởi một phân tử IgM hoàn chỉnh.',
 'C. tế bào plasma (tuơng bào, plasmocyte)',
 'D. IgM và IgG',
 'C. IgA và IgM',
 'D. đưa giải độc tố uốn ván vào cơ thể nhằm tạo đáp ứng miễn dịch chủ động chống uốn ván',
 'B. đưa kháng nguyên mầm bệnh vào cơ thể nhằm tạo ra trạng thái miễn dịch chủ động chống mầm bệnh khi mầm bệnh xâm nhập vào cơ thể được tiêm (hoặ',
 'A. glycoprotein',
 'C. kết hợp với vật lạ',
 'C. IgM',
 'C. IgM',
 'A. IgG']

In [10]:
dataset = generate_answers_batched(dataset, batch_size=4)
display(dataset.to_pandas())

Batch: 100%|██████████| 25/25 [00:35<00:00,  1.43s/it]


,Answer,ID,Question,Choices,Prediction
0,D. 10,2,Một phân tử IgM trong huyết thanh có mấy vị tr...,"[A. 1, B. 2, C. 5, D. 10]",C. 5
1,D. Tất cả đều đúng,3,Một phân tử IgM hoàn chỉnh trong huyết thanh c...,"[A. 4, B. 5, C. 10, D. Tất cả đều đúng]","B. 5\n\n\nĐể giải thích điều này, ta sẽ xem xé..."
2,"C. tế bào plasma (tuơng bào, plasmocyte)",4,Tế bào sản xuất kháng thể là,"[A. lympho bào B, B. lympho bào T, C. tế bào p...","C. tế bào plasma (tuơng bào, plasmocyte)"
3,C. IgG,5,Lớp kháng thể nào có thể đi qua được màng rau ...,"[A. IgM, B. IgA, C. IgG, D. IgM và IgG]",D. IgM và IgG
4,D. IgM,6,Kháng thể tự nhiên chống kháng nguyên hồng cầu...,"[A. IgG, B. IgG và IgA, C. IgA và IgM, D. IgM]",C. IgA và IgM
...,...,...,...,...,...
95,C. nhiễm virut,97,Hình thức đáp ứng miễn dịch qua trung gian tế ...,"[A. nhiễm vi khuẩn lao, B. nhiễm vi khuẩn tả, ...",A. nhiễm vi khuẩn lao
96,C. nhiễm virut,98,Hình thức đáp ứng miễn dịch qua trung gian tế ...,"[A. nhiễm vi khuẩn lao, B. nhiễm vi khuẩn tả, ...",A. nhiễm vi khuẩn lao
97,D. Typ IV: Quá mẫn trung gian tế bào,99,Phản ứng quá mẫn gây ra bệnh viêm da tiếp xúc ...,"[A. Typ I: Quá mẫn kiểu phản vệ, B. Typ II: Qu...",B. Typ II: Quá mẫn độc tế bào
98,A. Typ I: Quá mẫn kiểu phản vệ,100,Phù mặt diễn ra nhanh sau khi bị ong đốt thuôc...,"[A. Typ I: Quá mẫn kiểu phản vệ, B. Typ II: Qu...",B. Typ II: Quá mẫn độc tế bào


In [11]:
dataset[0]["Choices"]

['A. 1', 'B. 2', 'C. 5', 'D. 10']

In [12]:
fmt_acc = sum(
    pred.strip() in map(str.strip, choices)
    for pred, choices in zip(dataset["Prediction"], dataset["Choices"])
) / len(dataset)
ans_acc = sum(
    pred.strip() == ans.strip()
    for pred, ans in zip(dataset["Prediction"], dataset["Answer"])
) / len(dataset)
print(f"Format Accuracy: {fmt_acc}")
print(f"Answer Accuracy: {ans_acc}")

Format Accuracy: 0.93
Answer Accuracy: 0.35
